# Dependencies

## Installs

In [14]:
!pip install kaggle

  Using cached kaggle-1.6.17-py3-none-any.whl


## Imports

In [2]:
import os
import json
from datetime import datetime
import zipfile

In [3]:
import pandas as pd
import numpy as np

In [4]:
import boto3

s3 = boto3.resource('s3')

## Utilities

In [21]:
def create_lagged_features(df, lag_days, feature_columns):
    for feature in feature_columns:
        for lag in lag_days:
            df[f'{feature}_lag_{lag}'] = df.groupby('SecuritiesCode')[feature].shift(lag)
    return df

In [22]:
def create_rolling_features(df, window_sizes, feature_columns):
    for feature in feature_columns:
        for window in window_sizes:
            # Calculate rolling mean
            df[f'{feature}_rolling_mean_{window}'] = (
                df.groupby('SecuritiesCode')[feature]
                .transform(lambda x: x.rolling(window).mean())
            )
            # Calculate rolling standard deviation
            df[f'{feature}_rolling_std_{window}'] = (
                df.groupby('SecuritiesCode')[feature]
                .transform(lambda x: x.rolling(window).std())
            )
    return df

# Data

## Create an S3 Bucket to store the data

In [32]:
bucket_name = 'jpx-stock-prediction'
try:
    s3.create_bucket(Bucket=bucket_name)
    print(f'S3 Bucket {bucket_name} has been successfully created.')
except Exception as e:
    print(f'S3 Error:{e}')

S3 Bucket jpx-stock-prediction has been successfully created.


## Download the JPX Stock Exchange Dataset from Kaggle

In [16]:
os.makedirs(os.path.expanduser('./kaggle'), exist_ok=True)

In [17]:
# Move the kaggle.json file to the .kaggle directory
with open('kaggle.json', 'r') as f:
    kaggle_token = json.load(f)

with open(os.path.expanduser('./kaggle/kaggle.json'), 'w') as f:
    json.dump(kaggle_token, f)

# Set the permissions
os.chmod(os.path.expanduser('./kaggle/kaggle.json'), 0o600)

In [18]:
!kaggle competitions download -c jpx-tokyo-stock-exchange-prediction

 91%|█████████████████████████████████████▍   | 220M/241M [00:02<00:00, 160MB/s]
100%|█████████████████████████████████████████| 241M/241M [00:02<00:00, 114MB/s]


In [19]:
with zipfile.ZipFile('./jpx-tokyo-stock-exchange-prediction.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset-directory')

In [20]:
file_path = './dataset-directory/train_files/stock_prices.csv'
df_data = pd.read_csv(file_path)
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332531 entries, 0 to 2332530
Data columns (total 12 columns):
 #   Column            Dtype  
---  ------            -----  
 0   RowId             object 
 1   Date              object 
 2   SecuritiesCode    int64  
 3   Open              float64
 4   High              float64
 5   Low               float64
 6   Close             float64
 7   Volume            int64  
 8   AdjustmentFactor  float64
 9   ExpectedDividend  float64
 10  SupervisionFlag   bool   
 11  Target            float64
dtypes: bool(1), float64(7), int64(2), object(2)
memory usage: 198.0+ MB


## Extract, Load and Transform

In [23]:
features = ['Open', 'High', 'Low', 'Close', 'Volume']
lag_days = [1, 2, 3, 5, 10]
window_sizes = [5, 10, 20]

In [24]:
df_data.drop(axis=1,columns=['RowId','AdjustmentFactor','ExpectedDividend','SupervisionFlag'],inplace=True)
df_data['Date'] = pd.to_datetime(df_data['Date'])
df_data = df_data.sort_values(by=['SecuritiesCode', 'Date'])

In [25]:
df_data = create_lagged_features(df_data, lag_days, features)
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2332531 entries, 0 to 2332530
Data columns (total 33 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Date            datetime64[ns]
 1   SecuritiesCode  int64         
 2   Open            float64       
 3   High            float64       
 4   Low             float64       
 5   Close           float64       
 6   Volume          int64         
 7   Target          float64       
 8   Open_lag_1      float64       
 9   Open_lag_2      float64       
 10  Open_lag_3      float64       
 11  Open_lag_5      float64       
 12  Open_lag_10     float64       
 13  High_lag_1      float64       
 14  High_lag_2      float64       
 15  High_lag_3      float64       
 16  High_lag_5      float64       
 17  High_lag_10     float64       
 18  Low_lag_1       float64       
 19  Low_lag_2       float64       
 20  Low_lag_3       float64       
 21  Low_lag_5       float64       
 22  Low_lag_10      float64

In [26]:
df_data_copy = df_data.copy()
df_data_copy = create_rolling_features(df_data_copy, window_sizes, features)
df_data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2332531 entries, 0 to 2332530
Data columns (total 63 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Date                    datetime64[ns]
 1   SecuritiesCode          int64         
 2   Open                    float64       
 3   High                    float64       
 4   Low                     float64       
 5   Close                   float64       
 6   Volume                  int64         
 7   Target                  float64       
 8   Open_lag_1              float64       
 9   Open_lag_2              float64       
 10  Open_lag_3              float64       
 11  Open_lag_5              float64       
 12  Open_lag_10             float64       
 13  High_lag_1              float64       
 14  High_lag_2              float64       
 15  High_lag_3              float64       
 16  High_lag_5              float64       
 17  High_lag_10             float64       
 18  Low_lag

In [27]:
df_data_copy['HL_ratio'] = df_data_copy['High'] / df_data_copy['Low']
df_data_copy['CO_ratio'] = df_data_copy['Close'] / df_data_copy['Open']
df_data_copy['Day_of_Week'] = df_data_copy['Date'].dt.dayofweek
df_data_copy['Week_of_Year'] = df_data_copy['Date'].dt.isocalendar().week
df_data_copy['Month'] = df_data_copy['Date'].dt.month
df_data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2332531 entries, 0 to 2332530
Data columns (total 68 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Date                    datetime64[ns]
 1   SecuritiesCode          int64         
 2   Open                    float64       
 3   High                    float64       
 4   Low                     float64       
 5   Close                   float64       
 6   Volume                  int64         
 7   Target                  float64       
 8   Open_lag_1              float64       
 9   Open_lag_2              float64       
 10  Open_lag_3              float64       
 11  Open_lag_5              float64       
 12  Open_lag_10             float64       
 13  High_lag_1              float64       
 14  High_lag_2              float64       
 15  High_lag_3              float64       
 16  High_lag_5              float64       
 17  High_lag_10             float64       
 18  Low_lag

In [28]:
df_data_features = df_data_copy.drop(columns=['Date', 'SecuritiesCode'])
df_data_features = df_data_features.dropna()

In [30]:
df_data_features = df_data_features[['Target'] + [col for col in df_data_features.columns if col != 'Target']]
df_data_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2222559 entries, 35435 to 2332530
Data columns (total 66 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Target                  float64
 1   Open                    float64
 2   High                    float64
 3   Low                     float64
 4   Close                   float64
 5   Volume                  int64  
 6   Open_lag_1              float64
 7   Open_lag_2              float64
 8   Open_lag_3              float64
 9   Open_lag_5              float64
 10  Open_lag_10             float64
 11  High_lag_1              float64
 12  High_lag_2              float64
 13  High_lag_3              float64
 14  High_lag_5              float64
 15  High_lag_10             float64
 16  Low_lag_1               float64
 17  Low_lag_2               float64
 18  Low_lag_3               float64
 19  Low_lag_5               float64
 20  Low_lag_10              float64
 21  Close_lag_1             float64


In [31]:
df_randomized = df_data_features.sample(frac=1, random_state=123)
train_data, test_data = np.split(df_randomized, [int(0.8*len(df_randomized))])
print(f"Train:{train_data.shape} \nTest:{test_data.shape}")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Train:(1778047, 66) 
Test:(444512, 66)


## Upload to S3

In [33]:
prefix = 'xgboost-builtin'

In [35]:
train_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name,prefix,'train','train.csv')
test_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name,prefix,'test','test.csv')

print(f'Train Path: {train_csv_path}')
print(f'Test Path: {test_csv_path}')

Train Path: s3://jpx-stock-prediction/xgboost-builtin/train/train.csv
Test Path: s3://jpx-stock-prediction/xgboost-builtin/test/test.csv


In [37]:
train_data.to_csv(train_csv_path, index=False, header=False)
test_data.to_csv(test_csv_path, index=False, header=False)